# Trabalho Prático 3

## Introdução

A resolução deste trabalho prático tem 3 objetivos principais:

- Criar uma classe **Python** que implemente o algoritmo de _Boneh & Venkatesan_ .
- Implementação de um esquema de assinaturas digitais _FALCON_ .
- Estudar gamas de valores de determinados parâmetros que tornem viável um ataque de inversão de chave pública ou inversão do criptograma utilizando redução de bases.

Este relatório está, desta forma, dividido em três partes, cada parte correspondente à resolução de um dos problemas e, além disso, está estruturado de forma a que o texto entre os _snippets_ de código seja suficientemente explicativo sobre a implementação e desenho da solução em cada um dos problemas.

## Algoritmo de Boneh & Venkatesan

Esta secção tem como propósito definir uma classe **Python** que implemente o algoritmo de _Boneh & Venkatesan_ , que explora o _Hidden Number Problem_ que, se bem sucedido, permite obter um segredo a partir de um conjunto de dados. Além disso, a secção está dividida em duas partes:

- A primeira parte define apenas o algoritmo de _Boneh & Venkatesan_ .
- A segunda parte, apelidade de teste, fornece os parâmetros necessários à instância do algoritmo para que ele descubra o segredo **s** a partir dos mesmos.

**CONVÉM DEFINIR UM BOCADO O HNP AQUI. DIZER O QUE É.**

### Definição do algoritmo de Boneh & Venkatesan

In [26]:

class BV:
    
    def __init__(self,u,x,k,p,l):
        # construir a matriz L , lambda e o target T
        self.param_lambda = 2^(k+1)
        self.L = self.param_lambda * p * matrix.identity(l)
        self.L = self.L.transpose()
        self.L = self.L.insert_row(l,zero_vector(l))
        self.L = self.L.transpose()
        temp_x = [self.param_lambda * i for i in x]
        temp_x.append(1)
        self.L = self.L.insert_row(l,temp_x)
        self.target = [self.param_lambda * i for i in u]
        self.target.append(0)
        
    
    def solve(self,x,p):
        # Resolver o CVP utilizando a abordagem de Boneh & Venkatesan
        #s = var("s")
        #y = [s * xi // p for xi in x]
        # Como preencher o vetor y sendo que isto é uma expressão ?
        print y
        return ''

### Teste ao algoritmo de Boneh & Venkatesan

In [28]:
# Estes parâmetros têm que ser substituídos pelos verdadeiros.
# x possui l elementos gerados aleatoriamente entre 0 e p-1.
# u possui l elementos, onde cada ui é igual ao inteiro representativo dos k bits mais significativos de s*xi

u = [3,3,3]
x = [4,5,6]
k = 2
p = 3
l = 3
bv_algo = BV(u,x,k,p,l)
#bv_algo.solve(x,p)

## Esquema de assinaturas FALCON

Esta secção tem como propósito definir uma classe **Python** que implemente o esquema de assinaturas **FALCON** e, nesse sentido, este esquema fornece uma operação de inicialização da instância e dos seus parâmetros, uma operação para assinatura de mensagens e uma operação para verificação de uma assinatura com base numa mensagem.

### Definição do esquema FALCON

In [ ]:
class FALCON:
    
    def __init__(self):
        
    def sign(self,msg):
        
    def verify(self,msg,sig):

### Teste ao esquema FALCON

3

## Estudo de parâmetros em ataques de redução de bases

## Conclusão

## Referências